In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 

In [2]:
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\717\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
train=pd.read_csv('train.csv')

In [4]:
len(train)

12792

In [13]:
train.head(3)

,newsdesk,section,subsection,material,headline,abstract,keywords,word_count,pub_date,is_popular,n_comments,uniqueID
0,Editorial,Opinion,NaN,Editorial,Protect Veterans From Fraud,Congress could do much more to protect America...,"['Veterans', 'For-Profit Schools', 'Financial ...",680,2020-01-01,1,186,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
1,Games,Crosswords & Games,NaN,News,‘It’s Green and Slimy’,Christina Iverson and Jeff Chen ring in the Ne...,['Crossword Puzzles'],931,2020-01-01,1,257,nyt://article/9edddb54-0aa3-5835-a833-d311a76f...
2,Science,Science,NaN,News,Meteor Showers in 2020 That Will Light Up Nigh...,"All year long, Earth passes through streams of...","['Meteors and Meteorites', 'Space and Astronom...",1057,2020-01-01,0,6,nyt://article/04bc90f0-b20b-511c-b5bb-3ce13194...


In [10]:
train['pub_date'] = pd.to_datetime(train['pub_date'])

In [11]:
train['pub_date'] = pd.to_datetime(train['pub_date'].dt.strftime('%m/%d/%Y'))

In [12]:
train['pub_date'].dtype

dtype('<M8[ns]')

In [39]:
#distribution=train.groupby(['section',"is_popular"]).count()
#distribution

In [40]:
#distribution2=distribution[distribution["uniqueID"]>200]
#distribution2

In [41]:
################## find similar articles based on keywords ####################################

In [14]:
train_opn_kw=train[['uniqueID','keywords',"is_popular",'pub_date']]

In [16]:
# keywords list to sentences
train_opn_kw['kwSentence']=train_opn_kw.keywords.apply(lambda x: ', '.join(x.strip("[]").replace("'","").split(", ")))

<ipython-input-16-c54f6892dcfe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_opn_kw['kwSentence']=train_opn_kw.keywords.apply(lambda x: ', '.join(x.strip("[]").replace("'","").split(", ")))


In [17]:
train_opn_kw.head(2)

,uniqueID,keywords,is_popular,pub_date,kwSentence
0,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...,"['Veterans', 'For-Profit Schools', 'Financial ...",1,2020-01-01,"Veterans, For-Profit Schools, Financial Aid (E..."
1,nyt://article/9edddb54-0aa3-5835-a833-d311a76f...,['Crossword Puzzles'],1,2020-01-01,Crossword Puzzles


In [18]:
# search for similar article
doc = train_opn_kw[train_opn_kw['kwSentence'].notna()].reset_index()
documents = [i for i in train_opn_kw['kwSentence']]

In [19]:
documents

['Veterans, For-Profit Schools, Financial Aid (Education), Frauds and Swindling, Colleges and Universities, Veterans Affairs Department, Federal Trade Commission, University of Phoenix, Career Education Corporation',
 'Crossword Puzzles',
 'Meteors and Meteorites, Space and Astronomy, Earth, Solar System',
 'Space and Astronomy, Moon, Eclipses, Seasons and Months, Solar System, Meteors and Meteorites',
 'Space and Astronomy, Private Spaceflight, Rocket Science and Propulsion, National Aeronautics and Space Administration, European Space Agency, Boeing Company, Blue Origin, OneWeb Inc, Space Exploration Technologies Corp, Virgin Galactic, Bezos, Jeffrey P, United Arab Emirates, India, Israel, Japan, China, Russia, Moon, Mars (Planet), Sun, Mars 2020 (Mars Rover), Satellites, International Space Station, Comets, Perseverance (Mars Rover), Emirates Mars Mission, Tianwen-1 (Mars Mission), Starlink Satellite Constellation (SpaceX)',
 'Iraq, Iran, United States, Demonstrations, Protests and 

In [20]:
def get_similarity(i):
    # create temporary dataframe to store similarity score in each loop
    df_t=train_opn_kw[['uniqueID','kwSentence','pub_date']].copy()
    #input data search term , documents will not change
    search_terms = doc['kwSentence'][i]
    
    # calculate similarity score and store it in the temporary df_t
    tokenizer =TfidfVectorizer(stop_words='english')
    train_vectors = tokenizer.fit_transform(documents)
    test_vectors = tokenizer.transform([search_terms])
    cosine_similarities = linear_kernel(test_vectors, train_vectors).flatten()
    document_scores = np.asarray([item.item() for item in cosine_similarities])
    df_t['similarity score']=document_scores
    #find the capa that has similary score>=0.5 and store in tt
    tt=df_t[(df_t['similarity score']>=0.85)&(df_t['similarity score']<=0.999)].sort_values(by='similarity score',ascending=False)
    #add searchcapa column in tt
    tt['searchArticleID']=np.asarray([doc['uniqueID'][i]]*len(tt))  #加这里
    tt['searchKW']=np.asarray([doc['kwSentence'][i]]*len(tt))
    tt['search_pub_date']=np.asarray([doc['pub_date'][i]]*len(tt))
    tt.columns=['matchArticleID','matchKW','match_pub_date','similarity score','searchArticleID','searchKW','search_pub_date']
    return tt

In [21]:
# form match article dataframe and similarity score
df = pd.DataFrame(columns = ['matchArticleID','matchKW','match_pub_date','similarity score','searchArticleID','searchKW','search_pub_date'])
for i in range(len(doc)):
    tt=get_similarity(i)
    #append what we got to the df dataframe
    df=pd.concat([df,tt])

In [22]:
df.head()

,matchArticleID,matchKW,match_pub_date,similarity score,searchArticleID,searchKW,search_pub_date
459,nyt://article/fdcc8749-0fb7-5718-8658-c3c34769...,"Crossword Puzzles, Puzzles",2020-01-11,0.948744,nyt://article/9edddb54-0aa3-5835-a833-d311a76f...,Crossword Puzzles,2020-01-01
460,nyt://article/21ca8b09-97f5-50af-9bac-be444b12...,"Crossword Puzzles, Puzzles",2020-01-11,0.948744,nyt://article/9edddb54-0aa3-5835-a833-d311a76f...,Crossword Puzzles,2020-01-01
1967,nyt://article/ff238d50-8e67-580b-b489-26ebca77...,"Crossword Puzzles, Puzzles",2020-02-12,0.948744,nyt://article/9edddb54-0aa3-5835-a833-d311a76f...,Crossword Puzzles,2020-01-01
3264,nyt://article/33bc5a90-c6b9-54fd-b788-3a574fc7...,"Crossword Puzzles, Puzzles",2020-03-11,0.948744,nyt://article/9edddb54-0aa3-5835-a833-d311a76f...,Crossword Puzzles,2020-01-01
2147,nyt://article/106125db-34ef-5ab6-b2f9-1387200d...,"Obesity, Weight, Diet and Nutrition, Food, Fas...",2020-02-17,0.858390,nyt://article/da74e3b3-f027-5b03-9052-0bc2e69a...,"Weight, Fasting, Diet and Nutrition",2020-01-01


In [48]:
df_sd=df[df["match_pub_date"]==df["search_pub_date"]]
df_sd

,matchArticleID,matchKW,match_pub_date,similarity score,searchArticleID,searchKW,search_pub_date
70,nyt://article/dced2ad6-012b-5a80-abc3-c555cb50...,"Presidential Election of 2020, United States P...",2020-01-02,0.879536,nyt://article/d5686346-4b09-56b4-89f3-fe1c9c29...,"Trump, Donald J, Presidential Election of 2020...",2020-01-02
27,nyt://article/d5686346-4b09-56b4-89f3-fe1c9c29...,"Trump, Donald J, Presidential Election of 2020...",2020-01-02,0.879536,nyt://article/dced2ad6-012b-5a80-abc3-c555cb50...,"Presidential Election of 2020, United States P...",2020-01-02
268,nyt://article/14d6e103-e65d-5ede-aad0-05968c4b...,"Trump, Donald J, Impeachment, Bolton, John R, ...",2020-01-07,0.935502,nyt://article/9ed6057d-a0ea-5b35-8ce8-ae1e175b...,"McConnell, Mitch, Senate, Trump, Donald J, Imp...",2020-01-07
266,nyt://article/9ed6057d-a0ea-5b35-8ce8-ae1e175b...,"McConnell, Mitch, Senate, Trump, Donald J, Imp...",2020-01-07,0.935502,nyt://article/14d6e103-e65d-5ede-aad0-05968c4b...,"Trump, Donald J, Impeachment, Bolton, John R, ...",2020-01-07
551,nyt://article/873c285b-9c53-539d-8766-c0d0a9aa...,"Health Insurance and Managed Care, United Stat...",2020-01-13,0.882505,nyt://article/0651ce8d-c358-5e21-932e-5c3390b3...,Patient Protection and Affordable Care Act (20...,2020-01-13
...,...,...,...,...,...,...,...
12738,nyt://article/825316c3-d0e2-547d-a863-ed8bc5a4...,"Biden, Joseph R Jr, Trump, Donald J, Debates (...",2020-09-30,0.927910,nyt://article/87d979c5-4181-5e77-bc3a-db3622ca...,"Presidential Election of 2020, Debates (Politi...",2020-09-30
12748,nyt://article/1c2da958-3f03-52a3-8d50-6cb9ad94...,"Debates (Political), Biden, Joseph R Jr, Trump...",2020-09-30,0.927910,nyt://article/87d979c5-4181-5e77-bc3a-db3622ca...,"Presidential Election of 2020, Debates (Politi...",2020-09-30
12785,nyt://article/aadacc29-68bf-58b2-b786-0edf3b3d...,"Presidential Election of 2020, United States P...",2020-09-30,0.927910,nyt://article/87d979c5-4181-5e77-bc3a-db3622ca...,"Presidential Election of 2020, Debates (Politi...",2020-09-30
12754,nyt://article/0d1b3b35-c21b-52b9-b977-b5c4f07e...,"Presidential Election of 2020, Debates (Politi...",2020-09-30,0.900649,nyt://article/87d979c5-4181-5e77-bc3a-db3622ca...,"Presidential Election of 2020, Debates (Politi...",2020-09-30


In [57]:
df_sd2=df_sd.groupby(by=["matchArticleID"]).count()
df_sd2

,matchKW,match_pub_date,similarity score,searchArticleID,searchKW,search_pub_date
matchArticleID,,,,,,
nyt://article/02fb5996-2538-5d8e-aac7-b6fe7b11ff63,1,1,1,1,1,1
nyt://article/0651ce8d-c358-5e21-932e-5c3390b31f21,1,1,1,1,1,1
nyt://article/072049c0-9a3a-53da-a38e-dc55d56bb926,1,1,1,1,1,1
nyt://article/0823bec8-a9c9-56d3-85a9-4b01994f37ae,2,2,2,2,2,2
nyt://article/099afce5-31e2-5f4a-81e7-73a6f3c7f74d,2,2,2,2,2,2
...,...,...,...,...,...,...
nyt://interactive/565b9bb0-f64f-5b21-9196-b1f5049b9cfc,3,3,3,3,3,3
nyt://interactive/a42050f5-eaf2-523c-8f0d-09d68c595757,1,1,1,1,1,1
nyt://interactive/c6ec5381-c3f8-5a71-b23a-a0d9b18c7237,1,1,1,1,1,1


In [58]:
df_sd2.columns

Index(['matchKW', 'match_pub_date', 'similarity score', 'searchArticleID',
       'searchKW', 'search_pub_date'],
      dtype='object')

In [79]:
df_sd3=df_sd2.iloc[:,0:1]
df_sd3

,matchKW
matchArticleID,
nyt://article/02fb5996-2538-5d8e-aac7-b6fe7b11ff63,1
nyt://article/0651ce8d-c358-5e21-932e-5c3390b31f21,1
nyt://article/072049c0-9a3a-53da-a38e-dc55d56bb926,1
nyt://article/0823bec8-a9c9-56d3-85a9-4b01994f37ae,2
nyt://article/099afce5-31e2-5f4a-81e7-73a6f3c7f74d,2
...,...
nyt://interactive/565b9bb0-f64f-5b21-9196-b1f5049b9cfc,3
nyt://interactive/a42050f5-eaf2-523c-8f0d-09d68c595757,1
nyt://interactive/c6ec5381-c3f8-5a71-b23a-a0d9b18c7237,1


In [76]:
#check
#df[df["matchArticleID"]=="nyt://article/0823bec8-a9c9-56d3-85a9-4b01994f37ae"]

In [80]:
df_sd3.columns=["SimilarCount"]
df_sd3

,SimilarCount
matchArticleID,
nyt://article/02fb5996-2538-5d8e-aac7-b6fe7b11ff63,1
nyt://article/0651ce8d-c358-5e21-932e-5c3390b31f21,1
nyt://article/072049c0-9a3a-53da-a38e-dc55d56bb926,1
nyt://article/0823bec8-a9c9-56d3-85a9-4b01994f37ae,2
nyt://article/099afce5-31e2-5f4a-81e7-73a6f3c7f74d,2
...,...
nyt://interactive/565b9bb0-f64f-5b21-9196-b1f5049b9cfc,3
nyt://interactive/a42050f5-eaf2-523c-8f0d-09d68c595757,1
nyt://interactive/c6ec5381-c3f8-5a71-b23a-a0d9b18c7237,1


In [81]:
df_sd3.reset_index(inplace=True)
#df_sd4 = df_sd3.rename(columns = {'index':'uniqueID'})
df_sd4.columns=["uniqueID","SimilarCount"]
df_sd4

,uniqueID,SimilarCount
0,nyt://article/02fb5996-2538-5d8e-aac7-b6fe7b11...,1
1,nyt://article/0651ce8d-c358-5e21-932e-5c3390b3...,1
2,nyt://article/072049c0-9a3a-53da-a38e-dc55d56b...,1
3,nyt://article/0823bec8-a9c9-56d3-85a9-4b01994f...,2
4,nyt://article/099afce5-31e2-5f4a-81e7-73a6f3c7...,2
...,...,...
151,nyt://interactive/565b9bb0-f64f-5b21-9196-b1f5...,3
152,nyt://interactive/a42050f5-eaf2-523c-8f0d-09d6...,1
153,nyt://interactive/c6ec5381-c3f8-5a71-b23a-a0d9...,1
154,nyt://interactive/e772dc43-3061-5e8c-b109-3ad5...,2


In [85]:
#put this competition number to original set
train2=train.merge(df_sd4, how='left', on='uniqueID')

In [86]:
#replace NAN with 0
train2['SimilarCount'] = train2['SimilarCount'].fillna(0)
train2.head()

,newsdesk,section,subsection,material,headline,abstract,keywords,word_count,pub_date,is_popular,n_comments,uniqueID,SimilarCount
0,Editorial,Opinion,NaN,Editorial,Protect Veterans From Fraud,Congress could do much more to protect America...,"['Veterans', 'For-Profit Schools', 'Financial ...",680,2020-01-01,1,186,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...,0.0
1,Games,Crosswords & Games,NaN,News,‘It’s Green and Slimy’,Christina Iverson and Jeff Chen ring in the Ne...,['Crossword Puzzles'],931,2020-01-01,1,257,nyt://article/9edddb54-0aa3-5835-a833-d311a76f...,0.0
2,Science,Science,NaN,News,Meteor Showers in 2020 That Will Light Up Nigh...,"All year long, Earth passes through streams of...","['Meteors and Meteorites', 'Space and Astronom...",1057,2020-01-01,0,6,nyt://article/04bc90f0-b20b-511c-b5bb-3ce13194...,0.0
3,Science,Science,NaN,Interactive Feature,Sync your calendar with the solar system,"Never miss an eclipse, a meteor shower, a rock...","['Space and Astronomy', 'Moon', 'Eclipses', 'S...",0,2020-01-01,0,2,nyt://interactive/5b58d876-9351-50af-9b41-a312...,0.0
4,Science,Science,NaN,News,"Rocket Launches, Trips to Mars and More 2020 S...",A year full of highs and lows in space just en...,"['Space and Astronomy', 'Private Spaceflight',...",1156,2020-01-01,0,25,nyt://article/bd8647b3-8ec6-50aa-95cf-2b81ed12...,0.0


In [88]:
train2.to_csv("train2.csv")

# add to test set

In [89]:
test=pd.read_csv('test.csv')

In [90]:
test['pub_date'] = pd.to_datetime(test['pub_date'])

In [91]:
test['pub_date'] = pd.to_datetime(test['pub_date'].dt.strftime('%m/%d/%Y'))

In [92]:
test['pub_date'].dtype

dtype('<M8[ns]')

In [93]:
test_opn_kw=test[['uniqueID','keywords',"is_popular",'pub_date']]

In [94]:
# keywords list to sentences
test_opn_kw['kwSentence']=test_opn_kw.keywords.apply(lambda x: ', '.join(x.strip("[]").replace("'","").split(", ")))

<ipython-input-94-6784177bdabb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_opn_kw['kwSentence']=test_opn_kw.keywords.apply(lambda x: ', '.join(x.strip("[]").replace("'","").split(", ")))


In [95]:
test_opn_kw.head(2)

,uniqueID,keywords,is_popular,pub_date,kwSentence
0,nyt://article/e467c2ae-2df3-5836-a6ca-b23d0d33...,"['Presidential Election of 2020', 'Biden, Jose...",1,2020-10-01,"Presidential Election of 2020, Biden, Joseph R..."
1,nyt://article/9a7ef9e0-1334-56b2-a7f1-288c4887...,"['Presidential Election of 2020', 'United Stat...",1,2020-10-01,"Presidential Election of 2020, United States P..."


In [96]:
# search for similar article
doc = test_opn_kw[test_opn_kw['kwSentence'].notna()].reset_index()
documents = [i for i in test_opn_kw['kwSentence']]

In [97]:
documents

['Presidential Election of 2020, Biden, Joseph R Jr, Trump, Donald J, Debates (Political)',
 'Presidential Election of 2020, United States Politics and Government, Right-Wing Extremism and Alt-Right, Fringe Groups and Movements, Whites, Debates (Political), Demonstrations, Protests and Riots, Trump, Donald J, United States',
 'Black People, Blacks, Presidential Election of 2020, United States Politics and Government, State Legislatures, Elections, Senate, Democratic Party, Republican Party, Senate, Espy, Mike, Mississippi',
 'Crossword Puzzles',
 'Football, New England Patriots, Kansas City Chiefs, Los Angeles Chargers, Tampa Bay Buccaneers, Indianapolis Colts, Chicago Bears, Buffalo Bills, Las Vegas Raiders, Tennessee Titans, Pittsburgh Steelers, Mahomes, Patrick (1995- ), Baltimore Ravens',
 '',
 'Presidential Election of 2020, Debates (Political), Voting and Voters, Democratic Party, Republican Party, Trump, Donald J, Biden, Joseph R Jr',
 'Television, The Salisbury Poisonings (TV P

In [99]:
def get_similarity(i):
    # create temporary dataframe to store similarity score in each loop
    df_t=test_opn_kw[['uniqueID','kwSentence','pub_date']].copy()
    #input data search term , documents will not change
    search_terms = doc['kwSentence'][i]
    
    # calculate similarity score and store it in the temporary df_t
    tokenizer =TfidfVectorizer(stop_words='english')
    train_vectors = tokenizer.fit_transform(documents)
    test_vectors = tokenizer.transform([search_terms])
    cosine_similarities = linear_kernel(test_vectors, train_vectors).flatten()
    document_scores = np.asarray([item.item() for item in cosine_similarities])
    df_t['similarity score']=document_scores
    #find the capa that has similary score>=0.5 and store in tt
    tt=df_t[(df_t['similarity score']>=0.85)&(df_t['similarity score']<=0.999)].sort_values(by='similarity score',ascending=False)
    #add searchcapa column in tt
    tt['searchArticleID']=np.asarray([doc['uniqueID'][i]]*len(tt))  #加这里
    tt['searchKW']=np.asarray([doc['kwSentence'][i]]*len(tt))
    tt['search_pub_date']=np.asarray([doc['pub_date'][i]]*len(tt))
    tt.columns=['matchArticleID','matchKW','match_pub_date','similarity score','searchArticleID','searchKW','search_pub_date']
    return tt

In [100]:
# form match article dataframe and similarity score
df = pd.DataFrame(columns = ['matchArticleID','matchKW','match_pub_date','similarity score','searchArticleID','searchKW','search_pub_date'])
for i in range(len(doc)):
    tt=get_similarity(i)
    #append what we got to the df dataframe
    df=pd.concat([df,tt])

In [101]:
df.head()

,matchArticleID,matchKW,match_pub_date,similarity score,searchArticleID,searchKW,search_pub_date
1049,nyt://interactive/8f849572-045b-5f8f-b646-6bce...,"Debates (Political), Presidential Election of ...",2020-10-23,0.869443,nyt://article/e467c2ae-2df3-5836-a6ca-b23d0d33...,"Presidential Election of 2020, Biden, Joseph R...",2020-10-01
671,nyt://article/83a85cc7-edfd-5bb4-a774-5a566c9d...,"Biden, Joseph R Jr, Trump, Donald J, President...",2020-10-16,0.862302,nyt://article/e467c2ae-2df3-5836-a6ca-b23d0d33...,"Presidential Election of 2020, Biden, Joseph R...",2020-10-01
776,nyt://article/574b7d51-fe98-5cdb-b823-52ee603f...,"Coronavirus Reopenings, Coronavirus (2019-nCoV...",2020-10-19,0.889024,nyt://article/5164608b-f0eb-5807-9173-e0c5d471...,"Coronavirus Reopenings, Education (K-12), Teac...",2020-10-01
2609,nyt://article/e9175bde-34be-5d28-8c50-5c3699e8...,"Education (K-12), New York City, Coronavirus R...",2020-11-29,0.867872,nyt://article/5164608b-f0eb-5807-9173-e0c5d471...,"Coronavirus Reopenings, Education (K-12), Teac...",2020-10-01
2292,nyt://article/30ab7032-07be-5ba4-aaa6-17d1feb4...,"Customs, Etiquette and Manners",2020-11-19,0.872382,nyt://article/e037992a-8d31-5915-a65b-bf945e2d...,"Customs, Etiquette and Manners, Content Type: ...",2020-10-01


In [102]:
df_sd=df[df["match_pub_date"]==df["search_pub_date"]]
df_sd

,matchArticleID,matchKW,match_pub_date,similarity score,searchArticleID,searchKW,search_pub_date
83,nyt://article/6ba2e6d3-f4e0-59df-b6ff-5fa85011...,"Biden, Joseph R Jr, Presidential Election of 2...",2020-10-02,0.937555,nyt://article/251da920-4853-5926-9209-f5c47b39...,"Trump, Donald J, Coronavirus (2019-nCoV), Pres...",2020-10-02
84,nyt://article/2f26c4f3-c5ba-50ec-9a0d-0d74c10f...,"Trump, Donald J, Presidential Election of 2020...",2020-10-02,0.879700,nyt://article/251da920-4853-5926-9209-f5c47b39...,"Trump, Donald J, Coronavirus (2019-nCoV), Pres...",2020-10-02
86,nyt://article/67fbf62c-16d5-5703-8e59-6b7c9fd9...,"Trump, Donald J, Coronavirus (2019-nCoV), Pres...",2020-10-02,0.879700,nyt://article/251da920-4853-5926-9209-f5c47b39...,"Trump, Donald J, Coronavirus (2019-nCoV), Pres...",2020-10-02
84,nyt://article/2f26c4f3-c5ba-50ec-9a0d-0d74c10f...,"Trump, Donald J, Presidential Election of 2020...",2020-10-02,0.927458,nyt://article/cb3b825a-f27c-5403-84a3-ff5e815e...,"Presidential Election of 2020, Trump, Donald J...",2020-10-02
86,nyt://article/67fbf62c-16d5-5703-8e59-6b7c9fd9...,"Trump, Donald J, Coronavirus (2019-nCoV), Pres...",2020-10-02,0.927458,nyt://article/cb3b825a-f27c-5403-84a3-ff5e815e...,"Presidential Election of 2020, Trump, Donald J...",2020-10-02
75,nyt://article/5696c056-f681-5cfd-9cb2-7ba55385...,"Coronavirus (2019-nCoV), Trump, Donald J, Unit...",2020-10-02,0.874646,nyt://article/cb3b825a-f27c-5403-84a3-ff5e815e...,"Presidential Election of 2020, Trump, Donald J...",2020-10-02
84,nyt://article/2f26c4f3-c5ba-50ec-9a0d-0d74c10f...,"Trump, Donald J, Presidential Election of 2020...",2020-10-02,0.943058,nyt://article/5696c056-f681-5cfd-9cb2-7ba55385...,"Coronavirus (2019-nCoV), Trump, Donald J, Unit...",2020-10-02
86,nyt://article/67fbf62c-16d5-5703-8e59-6b7c9fd9...,"Trump, Donald J, Coronavirus (2019-nCoV), Pres...",2020-10-02,0.943058,nyt://article/5696c056-f681-5cfd-9cb2-7ba55385...,"Coronavirus (2019-nCoV), Trump, Donald J, Unit...",2020-10-02
73,nyt://article/cb3b825a-f27c-5403-84a3-ff5e815e...,"Presidential Election of 2020, Trump, Donald J...",2020-10-02,0.874646,nyt://article/5696c056-f681-5cfd-9cb2-7ba55385...,"Coronavirus (2019-nCoV), Trump, Donald J, Unit...",2020-10-02
56,nyt://article/251da920-4853-5926-9209-f5c47b39...,"Trump, Donald J, Coronavirus (2019-nCoV), Pres...",2020-10-02,0.937555,nyt://article/6ba2e6d3-f4e0-59df-b6ff-5fa85011...,"Biden, Joseph R Jr, Presidential Election of 2...",2020-10-02


In [103]:
df_sd2=df_sd.groupby(by=["matchArticleID"]).count()
df_sd2

,matchKW,match_pub_date,similarity score,searchArticleID,searchKW,search_pub_date
matchArticleID,,,,,,
nyt://article/0aefcafd-2915-5103-8d86-9e6c970d17b2,1,1,1,1,1,1
nyt://article/205e1c29-cc99-59b2-a21a-5faceaaaadd3,1,1,1,1,1,1
nyt://article/251da920-4853-5926-9209-f5c47b39c3ba,3,3,3,3,3,3
nyt://article/2c7bc6bb-c381-59a7-b378-734557c108c2,1,1,1,1,1,1
nyt://article/2f26c4f3-c5ba-50ec-9a0d-0d74c10fa5d1,4,4,4,4,4,4
nyt://article/312b302e-141b-5ed0-9d5d-53da66e828e9,1,1,1,1,1,1
nyt://article/33e661ec-1d4d-5f1c-b3bc-623af718eaa0,1,1,1,1,1,1
nyt://article/3768341e-86bd-50ed-acb1-2a5c873ca397,1,1,1,1,1,1
nyt://article/39c9e450-ef42-5a61-abb2-4a3dd0c96232,1,1,1,1,1,1


In [104]:
df_sd2.columns

Index(['matchKW', 'match_pub_date', 'similarity score', 'searchArticleID',
       'searchKW', 'search_pub_date'],
      dtype='object')

In [111]:
df_sd3=df_sd2.iloc[:,0:1]
df_sd3

,matchKW
matchArticleID,
nyt://article/0aefcafd-2915-5103-8d86-9e6c970d17b2,1
nyt://article/205e1c29-cc99-59b2-a21a-5faceaaaadd3,1
nyt://article/251da920-4853-5926-9209-f5c47b39c3ba,3
nyt://article/2c7bc6bb-c381-59a7-b378-734557c108c2,1
nyt://article/2f26c4f3-c5ba-50ec-9a0d-0d74c10fa5d1,4
nyt://article/312b302e-141b-5ed0-9d5d-53da66e828e9,1
nyt://article/33e661ec-1d4d-5f1c-b3bc-623af718eaa0,1
nyt://article/3768341e-86bd-50ed-acb1-2a5c873ca397,1
nyt://article/39c9e450-ef42-5a61-abb2-4a3dd0c96232,1


In [112]:
df_sd3.columns=["SimilarCount"]
df_sd3

,SimilarCount
matchArticleID,
nyt://article/0aefcafd-2915-5103-8d86-9e6c970d17b2,1
nyt://article/205e1c29-cc99-59b2-a21a-5faceaaaadd3,1
nyt://article/251da920-4853-5926-9209-f5c47b39c3ba,3
nyt://article/2c7bc6bb-c381-59a7-b378-734557c108c2,1
nyt://article/2f26c4f3-c5ba-50ec-9a0d-0d74c10fa5d1,4
nyt://article/312b302e-141b-5ed0-9d5d-53da66e828e9,1
nyt://article/33e661ec-1d4d-5f1c-b3bc-623af718eaa0,1
nyt://article/3768341e-86bd-50ed-acb1-2a5c873ca397,1
nyt://article/39c9e450-ef42-5a61-abb2-4a3dd0c96232,1


In [113]:
df_sd3.reset_index(inplace=True)
df_sd3.columns=["uniqueID","SimilarCount"]
df_sd3

,uniqueID,SimilarCount
0,nyt://article/0aefcafd-2915-5103-8d86-9e6c970d...,1
1,nyt://article/205e1c29-cc99-59b2-a21a-5faceaaa...,1
2,nyt://article/251da920-4853-5926-9209-f5c47b39...,3
3,nyt://article/2c7bc6bb-c381-59a7-b378-734557c1...,1
4,nyt://article/2f26c4f3-c5ba-50ec-9a0d-0d74c10f...,4
5,nyt://article/312b302e-141b-5ed0-9d5d-53da66e8...,1
6,nyt://article/33e661ec-1d4d-5f1c-b3bc-623af718...,1
7,nyt://article/3768341e-86bd-50ed-acb1-2a5c873c...,1
8,nyt://article/39c9e450-ef42-5a61-abb2-4a3dd0c9...,1
9,nyt://article/4f082583-65cd-517e-ba1d-6e4127b4...,1


In [115]:
#put this competition number to original set
test2=test.merge(df_sd3, how='left', on='uniqueID')

In [116]:
#replace NAN with 0
test2['SimilarCount'] = test2['SimilarCount'].fillna(0)
test2.head()

,newsdesk,section,subsection,material,headline,abstract,keywords,word_count,pub_date,is_popular,uniqueID,SimilarCount
0,OpEd,Opinion,NaN,Op-Ed,Anyone Else Want to See Trump ‘Shut Up’?,Our president as a terrible toddler.,"['Presidential Election of 2020', 'Biden, Jose...",925,2020-10-01,1,nyt://article/e467c2ae-2df3-5836-a6ca-b23d0d33...,0.0
1,OpEd,Opinion,NaN,Op-Ed,Trump Calls on Extremists to ‘Stand By’,"Instead of condemning violent groups, the pres...","['Presidential Election of 2020', 'United Stat...",902,2020-10-01,1,nyt://article/9a7ef9e0-1334-56b2-a7f1-288c4887...,0.0
2,OpEd,Opinion,NaN,Op-Ed,"Can Mike Espy Make History, Again?",If the Democratic Party claims to value Black ...,"['Black People', 'Blacks', 'Presidential Elect...",1412,2020-10-01,1,nyt://article/4bb2b763-0088-5e10-b204-19e404f7...,0.0
3,Games,Crosswords & Games,NaN,News,In Which Rikishi Wear Mawashi,Adam Fromm is on the line.,['Crossword Puzzles'],849,2020-10-01,1,nyt://article/0d96205f-edb8-5f1f-8c44-1ddf6ed5...,0.0
4,Sports,Sports,Pro Football,News,N.F.L. Week 4 Predictions: Our Picks Against t...,Tom Brady and the Buccaneers are building mome...,"['Football', 'New England Patriots', 'Kansas C...",2690,2020-10-01,0,nyt://article/afc8295b-3c22-5a5f-9539-3f77b7b8...,0.0


In [118]:
test2.to_csv("test2.csv")